# Cloud Front with API Gateway Origin

This notebook can be used to configure an active-standby two region serverless API project. This includes the 
following:

* Creation of a common API key that can be installed in both regions. This is needed to ensure transparent
failover from the perspective of the service consumer.

## Library Code

In [ ]:
# SDK Imports
import boto3

cformation_east = boto3.client('cloudformation', region_name='us-east-1')
cformation_west = boto3.client('cloudformation', region_name='us-west-2')

gw_east = boto3.client('apigateway', region_name='us-east-1')
gw_west = boto3.client('apigateway', region_name='us-west-2')

In [ ]:
def get_stack_name(service, stage):
    return '{}-{}'.format(service,stage)

In [ ]:
def get_endpoint(cf_client, stack_name):
    response = cf_client.describe_stacks(
        StackName=stack_name
    )
    
    outputs = response['Stacks'][0]['Outputs']
    endpoint =  [d for d in outputs if d['OutputKey'] == 'ServiceEndpoint'][0]['OutputValue']
    return endpoint

In [ ]:
def get_plan_and_api_ids(gw_client, service, stage):
    response = gw_client.get_usage_plans()
    plans = response['items']
    stack_name = get_stack_name(service, stage)
    plan =  [d for d in plans if d['name'] == stack_name][0]
    plan_id = plan['id']
    api_stage = [d for d in plan['apiStages'] if d['stage'] == stage][0]
    api_id = api_stage['apiId']
    return plan_id, api_id
    

In [ ]:
import uuid

def generate_api_key():
    return str(uuid.uuid4())

In [ ]:
def create_api_key_and_add_to_plan(gw_client, key_name, key_val, plan_id):
   
    create_key_response = gw_client.create_api_key(
        name=key_name,
        enabled=True,
        generateDistinctId=True,
        value=key_val
    )
    
    key_id = create_key_response['id']
    
    plan_key_response = gw_client.create_usage_plan_key(
        usagePlanId=plan_id,
        keyId=key_id,
        keyType='API_KEY'
    )
    
    return id, key_id

In [ ]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [ ]:
# Create a key and add it to the usage plan?
# - create_api_key - need key id output
# - you can get the usage plan id and the api id via get_usage_plan and matching the plan with same name
#   as the stack
# - create_usage_plan_key associates the key to the plan: inputs are plan id, key id

## Application Context

In [ ]:
service = 'serverless-rest-api-with-dynamodb'
stage = 's1'
cross_region_key_name = 'xregion_key'
api_stack_name = 'cf-api-todo'
bucket_name = 'xtds-cf-templates'
primary_region = 'us-east-1'

In [ ]:
stack_name = get_stack_name(service, stage)
east_endpoint = get_endpoint(cformation_east, stack_name)
print east_endpoint

west_endpoint = get_endpoint(cformation_west, stack_name)
print west_endpoint

## Key Synchronization

This part of the notebook creates a common key for the gateway fronted app in both regions.

In [ ]:
key_val = generate_api_key()
print key_val

In [ ]:
# Create east key and add to plan
plan_id_east, api_id_east = get_plan_and_api_ids(gw_east, service, stage)
key_val_east, key_id_east = create_api_key_and_add_to_plan(gw_east, cross_region_key_name, key_val, plan_id_east)

In [ ]:
plan_id_west, api_id_west = get_plan_and_api_ids(gw_west, service, stage)
key_val_west, key_id_west = create_api_key_and_add_to_plan(gw_west, cross_region_key_name, key_val, plan_id_west)

## Custom Domain Names

Now that API gateway deployments can be tagged as regional, we are free from the tyranny of cloud front certificate
restrictions that prevented us from registering certificates with the same domain name in two different regions.

With regional API deployments, we can associated the same SSL cert with the endpoints in both regions, and use the certificate domain as the route 53 alias to define failover or weight policies (or any others we desire).

In [ ]:
domain_name = 'superapi.elcaro.net'

### East

In [ ]:
# We need to select the certificate assocaiated with out domain name
acm_client = boto3.client('acm')

In [ ]:
response = acm_client.list_certificates()

summaryList = response['CertificateSummaryList']
print summaryList

domain_cert = [x for x in summaryList if x['DomainName'] == domain_name][0]
print domain_cert

cert_arn = domain_cert['CertificateArn']
print cert_arn

In [ ]:
# Create the domain name
response = gw_east.create_domain_name(
    domainName=domain_name,
    regionalCertificateArn=cert_arn,
    endpointConfiguration={
        'types': [
            'REGIONAL'
        ]
    }
)

print response

In [ ]:
regional_domain_name = response['regionalDomainName']
print regional_domain_name

## Gateway as Cloud Front Origin

### Create Cloud Front Distribution

In [ ]:
# Config specific to cloud front
hosted_zone_name = 'elcaro.net.'
domain_name = '*.elcaro.net'
api_cname = 'superapi.elcaro.net'

In [ ]:
acmClient = boto3.client('acm')
response = acmClient.list_certificates()
print response, '\n'

certificateArn = ''

for c in response['CertificateSummaryList']:
    print c['DomainName']
    if c['DomainName'] == domain_name:
        certificateArn = c['CertificateArn']
        
if certificateArn == '':
    print 'No Certificate Available in this Region for {}'.format(domain_name)
else:
    print 'certificate arn for', domain_name, certificateArn

In [ ]:
from urlparse import urlparse
parsed_uri = urlparse(east_endpoint)
api_domain = parsed_uri.netloc
bucket_base = form_s3_url_prefix(primary_region) + '/' + bucket_name

In [ ]:


response = cformation_east.create_stack(
    StackName=api_stack_name,
    TemplateURL= bucket_base + '/cdn.yml',
    Parameters=[
        {
            'ParameterKey': 'APIEndpoint',
            'ParameterValue':api_domain
        },
        {
            'ParameterKey': 'APIStage',
            'ParameterValue':stage
        },
        {
            'ParameterKey': 'CName',
            'ParameterValue':api_cname
        },
        {
            'ParameterKey': 'CertificateArn',
            'ParameterValue': certificateArn
        }
    ]
)

print response

In [ ]:
print 'waiting on create of {}'.format(api_stack_name)
waiter = cformation_east.get_waiter('stack_create_complete')
waiter.wait(
    StackName=api_stack_name
)

print 'stack created'

In [ ]:
# Describe the cloud front stack
response = cformation_east.describe_stacks(
    StackName=api_stack_name
)


In [ ]:
# Extract the cloud front domain from the stack output
print response
outputs = response['Stacks'][0]['Outputs']
cf_domain =  [d for d in outputs if d['OutputKey'] == 'CFDomain'][0]['OutputValue']
print cf_domain

### Create the Route 53 for the CName

In [ ]:
# Now create a route 53 alias
response = cformation_east.create_stack(
    StackName=api_stack_name + '-r53',
    TemplateURL= bucket_base + '/route53alias.yml',
    Parameters=[
        {
            'ParameterKey': 'HostedZoneName',
            'ParameterValue':hosted_zone_name
        },
        {
            'ParameterKey': 'RecordSetDomainName',
            'ParameterValue':api_cname
        },
        {
            'ParameterKey': 'CloudFrontDomain',
            'ParameterValue':cf_domain
        }
    ]
)

print response

In [ ]:
print 'waiting on create of {}-r53'.format(api_stack_name)
waiter = cformation_east.get_waiter('stack_create_complete')
waiter.wait(
    StackName=api_stack_name + '-r53'
)

print 'stack created'

### Curl Against the CName

In [ ]:
# Curl away, chuckles...
list_uri = 'https://' + api_cname + '/todos/'
post_uri = list_uri
print list_uri

health_uri='https://' + api_cname + '/todos/health'
print health_uri

# Note - if you don't have key_val from above you can uncomment this and 
# set it directly.
#key_val = '965a5060-8674-4e25-ad8b-7fca24a05249'

In [ ]:
%%bash -s "$list_uri" "$key_val"
curl -H x-api-key:$2 $1

In [ ]:
%%bash -s "$list_uri" "$key_val"
curl -X POST -H x-api-key:$2 $1 --data '{ "text": "Research fear of clowns" }'

In [ ]:
%%bash -s "$health_uri"
curl $1

## Route 53 Health Check

In [ ]:
import boto3
import uuid

client = boto3.client('route53')


response = client.create_health_check(
    
    CallerReference=str(uuid.uuid4()),
    HealthCheckConfig={
        'Type':'HTTPS',
        'ResourcePath':'/todos/health',
        'FullyQualifiedDomainName':api_cname
    }
)

print response

In [ ]:
hc_id = response['HealthCheck']['Id']
print 'health check id: {}'.format(hc_id)

In [ ]:
# Now tag the health check name
tag_resp = client.change_tags_for_resource(
    ResourceType='healthcheck',
    ResourceId=hc_id,
    AddTags=[
        {
            'Key':'Name',
            'Value':api_cname
        },
    ]
)

print tag_resp

In [ ]:
hc_resp = client.get_health_check_status(
    HealthCheckId=hc_id
)

print hc_resp

## Cloud Formation Update - Region Route Away

This section shows how to update the api origin associated with the cloud front fronting the (cloud front
wrapping) the API endpoint.

In [ ]:
route_away_endpoint = west_endpoint

In [ ]:
from urlparse import urlparse
parsed_uri = urlparse(route_away_endpoint)
route_away_origin = parsed_uri.netloc
print route_away_origin

In [ ]:
response = cformation_east.update_stack(
    StackName=api_stack_name,
    UsePreviousTemplate=True,
    Parameters=[
        {
            'ParameterKey': 'APIEndpoint',
            'ParameterValue':route_away_origin
        },
        {
            'ParameterKey': 'APIStage',
            'UsePreviousValue':True
        },
        {
            'ParameterKey': 'CName',
            'UsePreviousValue':True
        },
        {
            'ParameterKey': 'CertificateArn',
            'UsePreviousValue': True
        }
    ]
)

print response

In [ ]:
print 'waiting on update of {}'.format(api_stack_name)
waiter = cformation_east.get_waiter('stack_update_complete')
waiter.wait(
    StackName=api_stack_name
)

print 'stack updated'

In [ ]:
# May need to invalidate the cache here.

## Clean Up

Clean up stuff  - useful while building this book

In [ ]:
def cleanup_key_and_plan(gw_client, key_id, plan_id):
    response = gw_client.delete_usage_plan_key(
        usagePlanId=plan_id,
        keyId=key_id
    )

    print response
    
    response = gw_client.delete_api_key(
        apiKey=key_id
    )

    print response

In [ ]:
cleanup_key_and_plan(gw_east, key_id_east, plan_id_east)
cleanup_key_and_plan(gw_west, key_id_west, plan_id_west)